# Make each level to each Curve CSV FILE

In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_begin = []
num_row = []
start = 3
number_of_corner = 4
f_1 = 'beginner_expert_processedData/beginner/beginner_'
f_3 = '.csv'
num_begin = 25
curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]

df_concat = pd.DataFrame()

for curve_num in range(start,number_of_corner):
#     print(num_row)
    for idx in range(1, num_begin+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()
        
        tmp = df.astype(float)
        tmp['level'] =0
        
        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
        num_row.append(np.size(tmpcorner,0)) 
        
        df_begin.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_begin[idx-1]])      
           
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_begin'+'.csv')
    df_concat = pd.DataFrame()
    df_begin = []
    


In [4]:
df_exp = []
f_1 = 'beginner_expert_processedData/expert/expert_'
f_3 = '.csv'
num_exp = 19

df_concat = pd.DataFrame()

for curve_num in range(start,number_of_corner):
    for idx in range(1, num_exp+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()

        tmp = df.astype(float)
        tmp['level'] =1

        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
        num_row.append(np.size(tmpcorner,0)) 

        df_exp.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_exp[idx-1]])
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_expert'+'.csv')
    df_concat = pd.DataFrame()
    df_exp = []

In [5]:
print(num_row)
sequence_length = max(num_row)
print(sequence_length)


[111, 98, 105, 88, 115, 105, 112, 95, 94, 108, 127, 140, 135, 99, 97, 86, 90, 111, 97, 78, 79, 95, 82, 79, 84, 87, 86, 86, 84, 81, 81, 111, 88, 80, 78, 77, 77, 84, 80, 77, 79, 79, 80, 84]
140


In [57]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import StandardScaler
import time

# import time

left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration','level']


#Hyper-parameters
num_epochs = 15
batches = 1
learning_rate = 0.005
input_size = len(left_column)-1 # left column except 'level'
output_size = 2 # Expert and Beginner
hidden_size = 128 # ?
num_layers = 1
num_begin_train = round(num_begin*0.75)*(number_of_corner-start)
num_exp_train = round(num_exp*0.75)*(number_of_corner-start)
num_begin_test = num_begin*(number_of_corner-start) - num_begin_train
num_exp_test = num_exp*(number_of_corner-start) - num_exp_train

print(num_begin_train, num_exp_train,num_begin_test,num_exp_test)
aug = 1
## Define GRU, Loss func and Optimizer
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
#         out, _ = self.gru(x, h0)
        out, _ = self.lstm(x, (h0,c0)) 
        out = out[:, -1, :]
        out = self.fc(out)
        return out

gru = GRU(input_size, hidden_size, num_layers, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)  

gru.fc.weight.data.fill_(1)
gru.fc.bias.data.fill_(1)
print(gru.fc.weight,gru.fc.bias)

## Data Processing
array_x = []
array_y = []
input_x = []
input_y = []
n_row = []

df_tmp_begin = pd.DataFrame() 
df_tmp_exp = pd.DataFrame() 
for curve_num in range(start,number_of_corner):
    df_tmp_begin = pd.concat([df_tmp_begin,pd.read_csv('cornerData/corner_'+str(curve_num+1)+'_begin.csv')])
    df_tmp_exp   = pd.concat([df_tmp_exp,pd.read_csv('cornerData/corner_'+str(curve_num+1)+'_expert.csv')])    
df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp], ignore_index=True) 
df_curve1 = df_curve1.loc[:,left_column]
df_curve1_saved = df_curve1.loc[:,left_column] # data backup
df_curve1.to_csv('cornerData/corner_'+'_dfcurve1'+'.csv')



# scaler = StandardScaler()
# scaler.fit(x)
# scaler.mean_
# x = scaler.transform(x)
# print(x_normal)
# # print(x)
# # print(x['Distance'])
# print(type(x))
# print(type(x_normal))

datum = df_curve1_saved
yyy = datum.pop('level')
left = left_column.remove('level')
for i in range(0,num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start)):
#     x = df_curve1_saved.loc[0:num_row[i]-1
    y = yyy.loc[0:num_row[i]-1]
    x_original = datum.loc[0:num_row[i]-1]
    
#     print(x_original)
#     print(num_row[i],y[0],x_original.iloc[-1,0])
    
    scaler = StandardScaler()
    scaler.fit(x_original)
    scaler.mean_
    x_normal = scaler.transform(x_original)
    x_normal = np.pad(x_normal,[(0,sequence_length-num_row[i]),(0,0)])
    print(x_normal.shape)
    
    
    
    
    x = pd.DataFrame(x_normal,columns=left)
#     print(datum)
#     print(i)
#     print(num_row)
#     print(num_row[i])
    datum.drop(range(0,num_row[i]),inplace=True)
    datum.reset_index(drop=True, inplace=True)
    yyy.drop(range(0,num_row[i]),inplace=True)
    yyy.reset_index(drop=True, inplace=True)
    # y = x.pop('level')
    
#     # DATA Augmentation
#     nan = pd.DataFrame(np.nan,columns=range(x.shape[1]),index=range(x.shape[0]))
#     alter = pd.concat([x,nan]).sort_index()
#     alter = alter.interpolate()
#     alter.reset_index(drop=True, inplace=True)
#     x_aug = alter[alter.index%2==1]

    #DATA PADDING
    
#     x = x.pad(limit = sequence_length)
#     print(x)
#     time.sleep(300)
    
    
    
    array_x.append(x)
#     array_x.append(x_aug)
    array_y.append(y)
#     array_y.append(y)

    
## Randomize sequence 
sequence = np.arange((num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug)
np.random.seed(10)
np.random.shuffle(sequence)

# sequence = [0,1,2,15,4,5,6,7,8,9,18,11,12,13,14,19,20,21,34,23,24,25,26,27,28,37,30,31,32,33,3,16,17,10,22,35,36,29]
print(sequence)

# # Data Augmentation
# num_row = pd.Series(num_row)
# num_row = num_row.repeat(2)
# # sequence = pd.Series(sequence)
# # sequence = sequence.repeat(2)
# num_row.reset_index(drop=True, inplace=True)
# # sequence.reset_index(drop=True, inplace=True)
# print(num_row, sequence)

for i in sequence:
    input_x.append(array_x[i])
    input_y.append(array_y[i])
#     n_row.append(num_row[i])
    n_row.append(sequence_length)




## Train 
loss_list = []
iteration_list = []
accuracy_list = []
test_list=[]
count = 0
# torch.backends.cudnn.benchmark = True
print((num_begin_train + num_exp_train)*aug)

# start = time.time()  # 시작 시간 저장

for epoch in range(num_epochs):
    for i in range(0,(num_begin_train + num_exp_train)*aug):
        
#         print(i)
#         print(len(input_x))
        # array type (numpy) 앞
        X = np.array(input_x[i])
#         print(X)
        X = X.reshape(-1,n_row[i],input_size)
#         print(X)
        Y = np.array(input_y[i])   
#         print(X,X.shape,Y[0])
#         time.sleep(300)
        # tensor type (pytorch)
        X = torch.from_numpy(X)
        X = X.float()
        Y = torch.tensor([Y[0]])
        Y = Y.type(torch.LongTensor)
#         Y = Y.float()
        
        optimizer.zero_grad()
        output = gru(X)
        loss = criterion(output, Y)
        
        # Backward and optimize
#         optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         count += 1
#         loss_list.append(loss.data)
#         iteration_list.append(count)
#         accuracy_list.append(accuracy)
#         print (f'Loss: {loss.item():.4f}')
    count += 1
    loss_list.append(loss.data)
    iteration_list.append(count)
    print (f'Epoch: [{epoch}/{num_epochs}]' f'Loss: {loss.item():.4f}')
## Test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for i in range((num_begin_train + num_exp_train)*aug, (num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug):
        
        # array type (numpy)
        print(i)
        X = np.array(input_x[i])
        X = X.reshape(-1,n_row[i],input_size)
        Y = np.array(input_y[i])   

        # tensor type (pytorch)
        X = torch.from_numpy(X)
        X = X.float()
        Y = torch.tensor([Y[0]])
        Y = Y.type(torch.LongTensor)
#         Y = Y.float()
        output = gru(X)
        _, predicted = torch.max(output.data, 1)
        n_samples += Y.size(0)
        n_correct += (predicted == Y).sum().item()
        print(Y, predicted)
        


    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {(num_begin_test + num_exp_test)*aug} test images: {acc} %')
#     print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간



19 14 6 5
Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [ ]:
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("RNN: Loss vs Number of iteration")
plt.show()
print(f'Accuracy of the network on the {num_begin_test + num_exp_test} test images: {acc} %')
# print(predicted, Y, output)
print(gru.fc.weight,gru.fc.bias)
# print(iteration_list)
# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("RNN: Accuracy vs Number of iteration")
# plt.savefig('graph.png')
# plt.show()

In [ ]:
food = 'bread'
vars()['cat'] = 123
print(cat)
print(type(cat))

In [ ]:
a = pd.DataFrame() 
print(a)

In [ ]:
s = pd.Series([0, 1, np.nan, 10])
s.interpolate()

In [ ]:
print(food)

In [ ]:
# print(x)
x.shape
nan = pd.DataFrame(np.nan,columns=range(x.shape[1]),index=range(x.shape[0]))
alter = pd.concat([x,nan]).sort_index()
alter = alter.interpolate()
alter.reset_index(drop=True, inplace=True)
# print(alter[alter.index%2==0])
# print(alter[alter.index%2==1])


In [ ]:
# from sklearn.preprocessing import StandardScaler
# print(x['Distance'])
# scaler = StandardScaler()
# print(scaler.fit(x))
# print(scaler.mean_)
# # print(scaler.transform(x))
# x_normal = scaler.transform(x)
# print(x_normal)
# # print(x)
# # print(x['Distance'])
# print(type(x))
# print(type(x_normal))

In [ ]:
# %reset -f
print(gru.fc.weight,gru.fc.bias)

In [ ]:
import pandas
a = np.array([1,2,3])
a = pd.DataFrame([1,2,3])
print(a)
# a.repeat(2)
# np.array(num_row).repeat(2)
# num_row = pd.Series(num_row)
# num_row.repeat(2)
print(num_row)
type(num_row)
abc = pd.Series(num_row)
abc.repeat(2)

In [ ]:
# num_begin_train = round(num_begin*0.75)*(number_of_corner-start)
# num_exp_train = round(num_exp*0.75)*(number_of_corner-start)
# num_begin_test = num_begin*(number_of_corner-start) - num_begin_train
# num_exp_test = num_exp*(number_of_corner-start) - num_exp_train
# # print(num_begin_train, num_exp_train,num_begin_test,num_exp_test)
# num_begin_train

In [55]:
# # print(x)
# xx = pd.DataFrame(np.nan,index=range(4),columns=range(4))
# print(xx)
# yy = pd.DataFrame([1,2,3])
# # yy=[1,2,3]
# print(yy)
# xx = xx.replace([0,1,2],1)
# print(xx)

# df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
#                    'B': [5, 6, 7, 8, 9],
#                    'C': ['a', 'b', 'c', 'd', 'e']})
# print(df)
# df.replace([0, 1, 2, 3], [1,2])
aaa = np.arange(16)
aaa = aaa.reshape((4,4))
print(aaa)
np.pad(aaa,[(0,2),(0,0)])
# print(x_normal,type(x_normal))
# np.pad(x_normal,1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0]])